In [0]:
from pyspark.sql.functions import col, lit, count,try_divide


# # # Step 1: Load raw data lazily
# # raw_df = spark.read.option("header", True).csv("dbfs:/mnt/raw/sales.csv")

# # # Step 2: Transformations (lazy)
# # filtered_df = raw_df.filter(raw_df["amount"] > 100)
# # converted_df = filtered_df.withColumn("amount_usd", filtered_df["amount"].cast("double") * 1.1)
# # summary_df = converted_df.groupBy("region").agg({"amount_usd": "sum"})

# # # Step 3: Cache intermediate result (optional)
# # summary_df.cache()

# # # Step 4: Action - show result
# # summary_df.show()

# # # Step 5: Action - save to Delta Lake
# # summary_df.write.format("delta").mode("overwrite").save("/mnt/curated/sales_summary")

# df = spark.read.csv('samples.nyctaxi.trips')

# display(dbutils.fs.ls("/FileStore/tables"))


In [0]:
# # List catalogs (if UC enabled)
# spark.sql("SHOW CATALOGS").show()

# # List databases/schemas in a catalog
# spark.sql("SELECT * FROM "samples.nyctaxi" LIMIT 10").show()
df = spark.read.table("samples.nyctaxi.trips")
display(df)
df.printSchema()


In [0]:
df.select("*").filter(col("trip_distance") == 0).show()

In [0]:

# df = df.cache()
s = df.select(
    col("tpep_pickup_datetime").alias("new_column_name"),
    (col("trip_distance") * 2).alias("doubled_value"),
    lit("constant_value").alias("static_column")
).limit(5)

f = df.filter(df["fare_amount"]>15).limit(5)
g = df.groupBy("pickup_zip").agg(count("*").alias("count"))
display(s)
display(f)
display(g)

In [0]:
transformed = (
    df.filter(df["fare_amount"]>15)
      .withColumn("fare_per_passenger", try_divide(col("fare_amount"),col("trip_distance")))
      .select("*")
)

transformed.explain(True)
display(transformed)

In [0]:
# dbutils.fs.ls("/Volumes")